# 读取腔标定

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程将介绍如何使用读取模拟功能进行标定读取腔参数的实验，本教程的大纲如下：
- 背景介绍
- 准备工作
- 标定读取腔的跃迁频率
- 标定色散频移与耦合强度
- 测量衰减速率
- 总结

## 背景介绍

在超导电路中，如果想获得某个量子比特的状态，我们可以通过测量与之耦合的读取腔来间接实现对量子比特状态的读取，其具体操作是，我们首先向该读取腔施加一个脉冲信号，然后探测并分析从读取腔反射的脉冲信号。由于反射脉冲的振幅和相位的变化是由与之耦合的量子比特的状态决定的，因此我们可以通过这个变化间接读出量子比特在某次测量中的结果是 “0” 还是 “1”。

在实验中，我们首先需要对读取腔的相关参数进行标定，本教程介绍如何使用量脉完成对于读取腔标定的模拟。

一个由读取腔和量子比特相互耦合的系统可以用色散区域（Dispersive regime）的 Jaynes-Cumming 模型来描述 \[1\]：

$$
\hat{H}_{\rm JC} = \omega_r \hat{a}^\dagger \hat{a} + \frac{1}{2}\omega_q \hat{\sigma}_z + \chi \hat{a}^\dagger \hat{a} \hat{\sigma}_z.
$$


其中 $\hat{a}$、$\hat{a}^\dagger$ 分别是读取腔的湮灭和产生算符，而 $\hat{\sigma}_z$ 是量子比特的泡利 Z 算符。系数 $\omega_r$ 和 $\omega_q$ 分别是读取腔和量子比特的裸态频率（裸态频率是没有耦合的系统的本征频率）。$\chi$ 是色散频移，可表示为：

$$
\chi = \frac{g^2 \alpha}{\Delta_{qr}(\Delta_{qr} + \alpha)}.
$$

其中，$\alpha$ 是量子比特的非谐性，$\Delta_{qr} = \omega_q - \omega_r$ 是量子比特和读取腔的失调， $g$ 是量子比特与读取腔的耦合强度。由哈密顿量 $\hat{H}_{\rm JC}$ 中的相互作用项 $\chi \hat{a}^\dagger \hat{a} \hat{\sigma}_z$ 可知，量子比特分别处于 $|0\rangle$ 和 $|1\rangle$ 的时候，对应的读取腔的跃迁频率相差了 $2\chi$。因此在实验中分别将量子比特制备到 $|0\rangle$ 态和 $|1\rangle$ 态，然后分别对读取腔进行频率扫描的操作，得到两个跃迁频率 $f_0$ 以及 $f_1$，并计算频率差 $2\chi$，就可以通过上式间接计算量子比特和读取腔的耦合强度 $g$。

我们还可以通过测量读取腔频率的谱线宽度 $\kappa$ 来确定其光子衰减速率。为了模拟读取腔与量子比特组成的系统与环境的相互作用，该系统的密度矩阵 $\hat{\rho}(t)$ 的时间演化由 Lindblad 主方程给出 \[3, 4\]：


$$
\frac{d \hat{\rho}(t)}{dt} = -i[\hat{H}(t), \hat{\rho}(t)] + \frac{\kappa}{2}[2 \hat{a} \hat{\rho}(t) \hat{a}^\dagger - \hat{\rho}(t) \hat{a}^\dagger \hat{a} - \hat{a}^\dagger \hat{a} \hat{\rho}(t)].
$$


实验中，我们可以用洛伦兹函数拟合读取腔的频谱图并得到线宽，就能得到衰减速率 $\kappa$。

这里，我们最后提取的可观测量是读取腔场模的两个正交分量 $\hat{X} = \frac{1}{2}(\hat{a}^\dagger + \hat{a})$ 和 $\hat{Y} = \frac{i}{2}(\hat{a}^\dagger - \hat{a})$。实验中通过对从读取腔反射的脉冲经过一系列信号处理，我们就可以得到与这两个正交分量相关的电压值 $V_I$ 和 $V_Q$。

本教程中，我们使用量脉的工具求解读取腔的动力学演化，以模拟实验中标定读取腔的操作。我们标定的目标参数有：量子比特分别在 $|0\rangle$ 和 $|1\rangle$ 的读取腔跃迁频率 $\omega_{r0}$ 和 $\omega_{r1}$，谱线宽度 $\kappa$ 和色散频移 $\chi$。

## 准备工作

为了运行该教程，我们首先需要从量脉（Quanlse）以及其它 python 库导入所需要的包。

In [ ]:
# Import tools from Quanlse
from Quanlse.Simulator.ReadoutSim3Q import readoutSim3Q
from Quanlse.Calibration.Readout import resonatorSpec, fitLorentzian, lorentzian

# Import tools from other python libraries
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
from math import pi

## 标定读取腔的跃迁频率

在本节中，我们分别标定量子比特处于基态和第一激发态时的读取腔的跃迁频率。首先我们创建一个含有读取腔信息的 `ReadoutModel` 类的一个实例 `readoutModel`，这里我们直接用预设好的 `readoutSim3Q()` 实例作为演示。 

In [ ]:
readoutModel = readoutSim3Q()  # Initialize a readoutModel object

为了得到读取腔的跃迁频率，我们设定读取脉冲在某一频率范围内对读取腔进行频率扫描。设置扫描频率的范围 `freqRange`，驱动振幅 `amp` 以及读取脉冲的持续时间 `duration`。

In [ ]:
freqRange = np.linspace(7.105, 7.125, 60) * 2 * pi  # the range of frequency to probe the resonator, in 2 pi GHz
amp = 0.0005 * (2 * pi)  # drive amplitude, in 2 pi GHz
duration = 1000  #  duration of the readout pulse, in nanoseconds

调用函数 `resonatorSpec` 来完成一次对与处于基态的量子比特耦合的读取腔的扫描，输入想模拟的读取腔序号 `onRes`、频率范围 `freqRange`、读取脉冲振幅 `amp` 以及持续时间 `duration`，并且设置 `qubitState` 为基态。

In [ ]:
vi0, vq0 = resonatorSpec(readoutModel=readoutModel, onRes=[0], freqRange=freqRange, 
                         amplitude=amp, duration=duration, qubitState='ground')

得到信号 $V_I$ 和 $V_Q$ 后，绘制 $V_I$ 或者 $V_Q$ 随驱动频率变化的曲线。

In [ ]:
idx0 = find_peaks(vq0[0], height=max(vq0[0]))[0]  # find the index of the transition frequency
w0 = freqRange[idx0][0]  # transition frequency
print(f'The resonator transition frequency with qubit in ground state is {(w0 / (2 * pi)).round(3)} GHz')

In [ ]:
plt.plot(freqRange / (2 * pi), np.array(vq0[0]))
plt.plot()
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.title('Readout resonator spectrum')
plt.vlines((freqRange / (2 * pi))[idx0], 0, max(vq0[0]), linestyles='dashed')
plt.show()

从上图的模拟结果可知，当量子比特处于基态时，读取腔的跃迁频率大约在 7.118 GHz 左右。接下来我们用同样的步骤确定当量子比特处于激发态时的读取腔跃迁频率：

In [ ]:
vi1, vq1 = resonatorSpec(readoutModel=readoutModel, onRes=[0], freqRange=freqRange, 
                         amplitude=amp, duration=duration, qubitState='excited')

In [ ]:
idx1 = find_peaks(vq1[0], height=max(vq1[0]))[0]
w1 = freqRange[idx1][0]
print(f'The resonator transition frequency with qubit in excited state is {(w1 / (2 * pi)).round(3)} GHz')

In [ ]:
plt.plot(freqRange / (2 * pi), np.array(vq1[0]))
plt.plot()
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.title('Readout resonator spectrum')
plt.vlines((freqRange / (2 * pi))[idx1], 0, max(vq1[0]), linestyles='dashed')
plt.show()

由频谱图可知，当量子比特处于第一激发态时，读取腔的跃迁频率大约在 7.112 GHz 左右。

## 标定色散频移与耦合强度

在上节中，我们通过标定得到跃迁频率$f_0$ 和 $f_1$，因此可以直接计算色散频移 $\chi$：

$$
\chi = \frac{|f_0 - f_1|}{2}. 
$$

In [ ]:
chi = abs(w0 - w1) / 2
print(f'The dispersive shift is {(chi * 1e3 / (2 * pi)).round(3)} MHz')

结合"背景介绍"章节中给出的 $\chi$ 关于其它已知参数的表达式，我们可以计算读取腔与量子比特的耦合强度：

$$
g = \sqrt{\frac{\chi\Delta_{qr}(\Delta_{qr}+\alpha)}{\alpha}}.
$$

从 `readoutModel` 中提取出模型中的理论参数，并且使用上式计算耦合强度 $g$。

In [ ]:
# Extract parameters from the model

wq = readoutModel.pulseModel.qubitFreq[0]  # qubit bare frequency
alpha = readoutModel.pulseModel.qubitAnharm[0]  # qubit anharmonicity
wr = (w0 + w1) / 2  # estimated resonator frequency
detuning = wq - wr  # qubit-resonator detuning

In [ ]:
# coupling strength calculation

def qrCoupling(chi, detuning, alpha):
    g = np.sqrt(abs(chi * detuning * (detuning + alpha) / alpha))
    return g

In [ ]:
gEst = qrCoupling(chi, detuning, alpha)  # Estimated qubit-resonator coupling strength

然后我们比较通过间接计算得到耦合强度与理论耦合强度 $g$ 的值。

In [ ]:
g = readoutModel.coupling[0]  # therotical qubit-resonator coupling strength 
print(f'Theoretical coupling strength is {g * 1e3 / (2 * pi)} MHz')
print(f'Estimated coupling strength is {(gEst * 1e3 / (2 * pi)).round(1)} MHz')

通过标定色散频移以及间接计算得到的读取腔与量子比特的耦合强度大概是 132.4 MHz，与理论值 134.0 MHz 基本吻合。

## 测量衰减速率

得到读取腔的频谱后，我们可以使用洛伦兹函数拟合该频谱，从而得到线宽来估计衰减速率 $\kappa$。这里，我们通过调用 `fitLorentzian` 函数，并且输入频率扫描范围和信号强度，得到拟合的频谱曲线，从而得到出谱线宽度 $\kappa$：

In [ ]:
param, cov = fitLorentzian(freqRange, vq0[0])  # Fit the curve using lorentzian function
kappaEst = abs(param[2])  # Estimated linewidth

In [ ]:
plt.plot(freqRange / (2 * pi), lorentzian(freqRange, param[0], param[1], param[2], param[3]), '.')
plt.plot(freqRange / (2 * pi), vq0[0])
plt.xlabel('$\omega_d$ (GHz)')
plt.ylabel('signal (a.u.)')
plt.title('Readout resonator spectrum')
plt.show()

比较衰减速率（或谱线宽度）的理论值和通过标定计算的值。

In [ ]:
kappa = readoutModel.dissipation

print(f'Theoretical decay rate is {kappa * 1e3 / (2 * pi)} MHz')
print(f'Estimated linewidth is {(kappaEst * 1e3 / (2 * pi)).round(3)} MHz')

从模拟结果可知，我们在主方程设定的衰减速率 $\kappa$ 是 2.0 MHz，而通过频谱得到的线宽是 1.987 MHz。这说明了在实验中通过对读取腔进行频率的扫描以及计算线宽，可以间接标定读取腔与环境相互作用的强度。

## 总结
用户可以通过点击这个链接 [tutorial-readout-cavity-calibration-cn.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-readout-cavity-calibration-cn.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面并获取相关代码以运行该程序。用户可以尝试不同的读取腔参数，运行该教程的代码以模拟超导量子计算实验中读取腔的校准。

## 参考文献

\[1\] [Blais, Alexandre, et al. "Cavity quantum electrodynamics for superconducting electrical circuits: An architecture for quantum computation." *Physical Review A* 69.6 (2004): 062320.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.062320)

\[2\] [Koch, Jens, et al. "Charge-insensitive qubit design derived from the Cooper pair box." *Physical Review A* 76.4 (2007): 042319.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.76.042319)

\[3\] [Lindblad, Goran. "On the generators of quantum dynamical semigroups." *Communications in Mathematical Physics* 48.2 (1976): 119-130.](https://link.springer.com/article/10.1007/bf01608499)

\[4\] [Bianchetti, R., et al. "Dynamics of dispersive single-qubit readout in circuit quantum electrodynamics." *Physical Review A* 80.4 (2009): 043840.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.80.043840)